# Notebook Completo - Análisis y Detección de Anomalías en Planta Solar
# Ejecutar las celdas en orden

# ========================================
# CELDA 1: Importación de librerías
# ========================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EmpiricalCovariance
from scipy.stats import chi2
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")


# ========================================
# CELDA 2: Carga de datos
# ========================================

In [ ]:
def load_solar_data():
    """Carga los tres archivos de datos solares"""
    try:
        # Cargar datos ambientales
        env_data = pd.read_csv('environment_data.csv', parse_dates=['measured_on'])
        print(f"✓ Datos ambientales: {env_data.shape}")
        
        # Cargar datos de irradiancia
        irr_data = pd.read_csv('irradiance_data.csv', parse_dates=['measured_on'])
        print(f"✓ Datos de irradiancia: {irr_data.shape}")
        
        # Cargar datos eléctricos
        elec_data = pd.read_csv('chunk_electrical_data.csv', parse_dates=['measured_on'])
        print(f"✓ Datos eléctricos: {elec_data.shape}")
        
        return env_data, irr_data, elec_data
    except Exception as e:
        print(f"Error cargando datos: {e}")
        return None, None, None

env_data, irr_data, elec_data = load_solar_data()

# ========================================
# CELDA 3: Exploración de datos
# ========================================

In [ ]:
def explore_data(df, name):
    """Exploración básica de cada dataset"""
    print(f"\n{'='*50}")
    print(f"ANÁLISIS DE: {name}")
    print(f"{'='*50}")
    
    # Info básica
    print(f"\n📊 Forma: {df.shape}")
    print(f"📅 Rango temporal: {df['measured_on'].min()} a {df['measured_on'].max()}")
    
    # Valores faltantes
    missing = df.isnull().sum()
    if missing.any():
        print(f"\n⚠️ Valores faltantes:")
        print(missing[missing > 0])
    else:
        print(f"\n✓ Sin valores faltantes")
    
    # Estadísticas descriptivas (excluyendo fecha)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"\n📈 Estadísticas descriptivas:")
    print(df[numeric_cols].describe().round(2))
    
    return df

# Explorar cada dataset
env_data = explore_data(env_data, "DATOS AMBIENTALES")
irr_data = explore_data(irr_data, "DATOS DE IRRADIANCIA")
elec_data = explore_data(elec_data, "DATOS ELÉCTRICOS")


# ========================================
# CELDA 4: Fusión y limpieza de datos
# ========================================

In [ ]:
def clean_and_merge_data(env_data, irr_data, elec_data):
    """
    Limpia y fusiona los datasets por fecha
    """
    print("\n🔧 PROCESAMIENTO DE DATOS...")
    
    # Eliminar duplicados por fecha si existen
    env_data = env_data.drop_duplicates(subset=['measured_on'])
    irr_data = irr_data.drop_duplicates(subset=['measured_on'])
    elec_data = elec_data.drop_duplicates(subset=['measured_on'])
    
    # Merge secuencial
    print("\n📊 Fusionando datasets...")
    
    # Primero: ambiental con irradiancia
    merged_data = pd.merge(env_data, irr_data, on='measured_on', how='inner')
    print(f"✓ Ambiente + Irradiancia: {merged_data.shape}")
    
    # Segundo: resultado con eléctrico
    merged_data = pd.merge(merged_data, elec_data, on='measured_on', how='inner')
    print(f"✓ Dataset final: {merged_data.shape}")
    
    # Ordenar por fecha
    merged_data = merged_data.sort_values('measured_on')
    
    # Manejar valores infinitos
    merged_data = merged_data.replace([np.inf, -np.inf], np.nan)
    
    # Imputar NaN con interpolación
    numeric_cols = merged_data.select_dtypes(include=[np.number]).columns
    merged_data[numeric_cols] = merged_data[numeric_cols].interpolate(method='linear', limit=5)
    
    # Eliminar filas con muchos NaN
    merged_data = merged_data.dropna(thresh=len(merged_data.columns)*0.8)
    
    print(f"\n✅ Dataset limpio final: {merged_data.shape}")
    
    return merged_data

# Fusionar y limpiar
merged_data = clean_and_merge_data(env_data, irr_data, elec_data)


# ========================================
# CELDA 5: Ingeniería de características
# ========================================

In [ ]:
def create_features(df):
    """
    Crea características derivadas relevantes para análisis solar
    """
    print("\n🔬 CREANDO CARACTERÍSTICAS DERIVADAS...")
    
    original_cols = len(df.columns)
    
    # Características temporales
    df['hour'] = df['measured_on'].dt.hour
    df['day_of_year'] = df['measured_on'].dt.dayofyear
    df['month'] = df['measured_on'].dt.month
    
    # Indicador día/noche (aproximado)
    df['is_daytime'] = ((df['hour'] >= 6) & (df['hour'] <= 18)).astype(int)
    
    # Potencia DC total de todos los inversores
    dc_power_cols = []
    for i in range(1, 25):
        col_name = f'dc_power_inv_{i:02d}'
        dc_current = f'inv_{i:02d}_dc_current_inv' 
        dc_voltage = f'inv_{i:02d}_dc_voltage_inv'
        
        # Buscar columnas que coincidan
        current_cols = [c for c in df.columns if dc_current in c]
        voltage_cols = [c for c in df.columns if dc_voltage in c]
        
        if current_cols and voltage_cols:
            df[col_name] = df[current_cols[0]] * df[voltage_cols[0]]
            dc_power_cols.append(col_name)
    
    # Potencia total DC y AC
    ac_power_cols = [col for col in df.columns if 'ac_power' in col]
    if dc_power_cols:
        df['total_dc_power'] = df[dc_power_cols].sum(axis=1)
    else:
        df['total_dc_power'] = 0
        
    df['total_ac_power'] = df[ac_power_cols].sum(axis=1)
    
    # Eficiencia del sistema
    df['system_efficiency'] = np.where(
        df['total_dc_power'] > 0,
        (df['total_ac_power'] / df['total_dc_power']) * 100,
        0
    )
    
    # Performance Ratio (PR) - rendimiento respecto a condiciones ideales
    df['performance_ratio'] = np.where(
        df['poa_irradiance_o_149574'] > 50,  # Solo cuando hay sol significativo
        df['total_ac_power'] / df['poa_irradiance_o_149574'],
        0
    )
    
    # Relación temperatura-eficiencia (pérdidas térmicas)
    df['temp_impact'] = df['ambient_temperature_o_149575'] * df['system_efficiency']
    
    # Indicadores de anomalía por inversor
    for i in range(1, 25):
        ac_col = [c for c in ac_power_cols if f'inv_{i:02d}' in c]
        if ac_col and len(ac_power_cols) > 0:
            # Desviación del inversor respecto a la media
            df[f'inv_{i:02d}_deviation'] = np.where(
                df[ac_power_cols].std(axis=1) > 0,
                (df[ac_col[0]] - df[ac_power_cols].mean(axis=1)) / df[ac_power_cols].std(axis=1),
                0
            )
    
    print(f"✅ Características creadas: {len(df.columns) - original_cols} nuevas")
    
    return df

# Crear características
featured_data = create_features(merged_data.copy())

# ========================================
# CELDA 6: Selección de variables relevantes
# ========================================

In [ ]:
def select_relevant_features(df):
    """
    Selecciona las variables más relevantes para detección de anomalías
    """
    print("\n🎯 SELECCIÓN DE CARACTERÍSTICAS RELEVANTES")
    
    # Variables clave para análisis de anomalías en plantas solares
    key_features = [
        # Ambientales
        'ambient_temperature_o_149575',
        'wind_speed_o_149576',
        'poa_irradiance_o_149574',
        
        # Rendimiento global
        'total_dc_power',
        'total_ac_power',
        'system_efficiency',
        'performance_ratio',
        
        # Temporales
        'hour',
        'month',
        'is_daytime'
    ]
    
    # Añadir desviaciones de inversores
    deviation_cols = [col for col in df.columns if 'deviation' in col]
    key_features.extend(deviation_cols[:5])  # Top 5 inversores con más variación
    
    # Verificar que existen
    key_features = [f for f in key_features if f in df.columns]
    
    print(f"\n📋 Variables seleccionadas ({len(key_features)}):")
    for i, feat in enumerate(key_features, 1):
        print(f"   {i}. {feat}")
    
    # Calcular correlaciones
    corr_matrix = df[key_features].corr()
    
    # Visualizar matriz de correlación
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                cmap='coolwarm', center=0, square=True, 
                linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlación - Variables Clave', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
    
    # Análisis de correlaciones importantes
    print("\n🔍 CORRELACIONES IMPORTANTES (|r| > 0.7):")
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr.append({
                    'Variable 1': corr_matrix.columns[i],
                    'Variable 2': corr_matrix.columns[j],
                    'Correlación': corr_matrix.iloc[i, j]
                })
    
    if high_corr:
        corr_df = pd.DataFrame(high_corr).sort_values('Correlación', 
                                                      key=abs, 
                                                      ascending=False)
        print(corr_df.to_string(index=False))
    
    return key_features, corr_matrix

# Seleccionar características
selected_features, correlation_matrix = select_relevant_features(featured_data)



# ========================================
# CELDA 7: Preparación de datos horarios
# ========================================

In [ ]:
def prepare_hourly_data(df, features):
    """
    Prepara datos con promedios horarios para detección de anomalías
    """
    print("\n⏰ PREPARANDO PROMEDIOS HORARIOS...")
    
    # Crear columna de hora redondeada
    df['hour_rounded'] = df['measured_on'].dt.floor('H')
    
    # Agrupar por hora y calcular promedios
    # Incluir hour_rounded en el groupby
    agg_dict = {feat: 'mean' for feat in features}
    hourly_data = df.groupby('hour_rounded').agg(agg_dict).reset_index()
    
    # Recalcular la columna hour si no está
    if 'hour' not in hourly_data.columns:
        hourly_data['hour'] = hourly_data['hour_rounded'].dt.hour
    
    # Filtrar solo horas diurnas para análisis solar
    hourly_data_day = hourly_data[
        (hourly_data['hour'] >= 6) & 
        (hourly_data['hour'] <= 18) &
        (hourly_data['poa_irradiance_o_149574'] > 50)  # Con irradiancia significativa
    ].copy()
    
    print(f"✅ Datos horarios preparados: {hourly_data_day.shape}")
    
    # Normalizar datos
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(hourly_data_day[features])
    
    return hourly_data_day, features_scaled, scaler

# Preparar datos
hourly_data, X_scaled, scaler = prepare_hourly_data(featured_data, selected_features)


# ========================================
# CELDA 8: Detector de Anomalías - Distancia Euclidiana
# ========================================

In [ ]:
def euclidean_anomaly_detector(X, data, contamination=0.05):
    """
    Detecta anomalías usando distancia euclidiana
    Marca el 5% más lejano como anomalías
    """
    print("\n🔴 DETECTOR 1: DISTANCIA EUCLIDIANA")
    print("="*50)
    
    # Calcular centroide
    centroid = np.mean(X, axis=0)
    
    # Calcular distancias euclidianas
    distances = np.sqrt(np.sum((X - centroid)**2, axis=1))
    
    # Determinar threshold (percentil 95)
    threshold = np.percentile(distances, (1 - contamination) * 100)
    
    # Identificar anomalías
    anomalies = distances > threshold
    
    # Añadir resultados al dataframe
    data['euclidean_distance'] = distances
    data['euclidean_anomaly'] = anomalies
    
    # Estadísticas
    print(f"📊 Estadísticas:")
    print(f"   - Distancia promedio: {np.mean(distances):.2f}")
    print(f"   - Distancia máxima: {np.max(distances):.2f}")
    print(f"   - Threshold (95%): {threshold:.2f}")
    print(f"   - Anomalías detectadas: {np.sum(anomalies)} ({np.sum(anomalies)/len(data)*100:.1f}%)")
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histograma de distancias
    ax1.hist(distances, bins=50, alpha=0.7, color='blue', edgecolor='black')
    ax1.axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold = {threshold:.2f}')
    ax1.set_xlabel('Distancia Euclidiana')
    ax1.set_ylabel('Frecuencia')
    ax1.set_title('Distribución de Distancias Euclidianas')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Serie temporal con anomalías
    ax2.scatter(data['hour_rounded'], data['total_ac_power'], 
                c=data['euclidean_anomaly'], cmap='coolwarm', 
                alpha=0.6, s=50)
    ax2.set_xlabel('Fecha')
    ax2.set_ylabel('Potencia AC Total (W)')
    ax2.set_title('Anomalías Detectadas - Distancia Euclidiana')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Fechas con anomalías
    anomaly_dates = data[data['euclidean_anomaly']]['hour_rounded'].tolist()
    
    print(f"\n📅 Primeras 10 fechas con anomalías:")
    for i, date in enumerate(anomaly_dates[:10], 1):
        print(f"   {i}. {date}")
    
    return data, anomaly_dates

# Aplicar detector euclidiano
hourly_data, euclidean_anomalies = euclidean_anomaly_detector(
    X_scaled, hourly_data.copy(), contamination=0.05
)

# ========================================
# CELDA 9: Detector de Anomalías - Distancia de Mahalanobis
# ========================================

In [ ]:
def mahalanobis_anomaly_detector(X, data, contamination=0.05):
    """
    Detecta anomalías usando distancia de Mahalanobis
    Considera las correlaciones entre variables
    """
    print("\n🟡 DETECTOR 2: DISTANCIA DE MAHALANOBIS")
    print("="*50)
    
    # Calcular media y covarianza
    mean = np.mean(X, axis=0)
    
    # Usar Minimum Covariance Determinant para robustez
    cov_estimator = EmpiricalCovariance()
    cov_estimator.fit(X)
    
    # Calcular distancias de Mahalanobis
    mahal_distances = cov_estimator.mahalanobis(X)
    
    # Determinar threshold usando distribución chi-cuadrado
    p = X.shape[1]
    threshold = chi2.ppf(1 - contamination, df=p)
    
    # Identificar anomalías
    anomalies = mahal_distances > threshold
    
    # Añadir resultados
    data['mahalanobis_distance'] = mahal_distances
    data['mahalanobis_anomaly'] = anomalies
    
    # Estadísticas
    print(f"📊 Estadísticas:")
    print(f"   - Dimensiones: {p} variables")
    print(f"   - Distancia promedio: {np.mean(mahal_distances):.2f}")
    print(f"   - Distancia máxima: {np.max(mahal_distances):.2f}")
    print(f"   - Threshold χ²(95%): {threshold:.2f}")
    print(f"   - Anomalías detectadas: {np.sum(anomalies)} ({np.sum(anomalies)/len(data)*100:.1f}%)")
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Q-Q plot para verificar distribución chi-cuadrado
    stats.probplot(mahal_distances, dist=stats.chi2, sparams=(p,), plot=ax1)
    ax1.set_title('Q-Q Plot: Mahalanobis vs Chi-cuadrado')
    ax1.grid(True, alpha=0.3)
    
    # Scatter plot: Euclidiana vs Mahalanobis
    ax2.scatter(data['euclidean_distance'], data['mahalanobis_distance'], 
                c=data['mahalanobis_anomaly'], cmap='viridis', 
                alpha=0.6, s=50)
    ax2.set_xlabel('Distancia Euclidiana')
    ax2.set_ylabel('Distancia de Mahalanobis')
    ax2.set_title('Comparación de Distancias')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Análisis de diferencias con Euclidiana
    both_anomalies = data['euclidean_anomaly'] & data['mahalanobis_anomaly']
    only_euclidean = data['euclidean_anomaly'] & ~data['mahalanobis_anomaly']
    only_mahalanobis = ~data['euclidean_anomaly'] & data['mahalanobis_anomaly']
    
    print(f"\n🔍 Comparación con Euclidiana:")
    print(f"   - Anomalías en ambos: {np.sum(both_anomalies)}")
    print(f"   - Solo Euclidiana: {np.sum(only_euclidean)}")
    print(f"   - Solo Mahalanobis: {np.sum(only_mahalanobis)}")
    
    # Fechas con anomalías
    anomaly_dates = data[data['mahalanobis_anomaly']]['hour_rounded'].tolist()
    
    print(f"\n📅 Primeras 10 fechas con anomalías (Mahalanobis):")
    for i, date in enumerate(anomaly_dates[:10], 1):
        print(f"   {i}. {date}")
    
    return data, anomaly_dates

# Aplicar detector Mahalanobis
hourly_data, mahalanobis_anomalies = mahalanobis_anomaly_detector(
    X_scaled, hourly_data.copy(), contamination=0.05
)


# ========================================
# CELDA 10: Detector de Anomalías - Isolation Forest
# ========================================


In [ ]:
def isolation_forest_detector(X, data, contamination=0.05):
    """
    Detecta anomalías usando Isolation Forest
    Algoritmo basado en árboles de decisión
    """
    print("\n🟢 DETECTOR 3: ISOLATION FOREST")
    print("="*50)
    
    # Configurar y entrenar Isolation Forest
    iso_forest = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_estimators=100,
        max_samples='auto',
        n_jobs=-1
    )
    
    # Predecir anomalías (-1 = anomalía, 1 = normal)
    predictions = iso_forest.fit_predict(X)
    anomaly_scores = iso_forest.score_samples(X)
    
    # Convertir a booleano (True = anomalía)
    anomalies = predictions == -1
    
    # Añadir resultados
    data['isolation_score'] = anomaly_scores
    data['isolation_anomaly'] = anomalies
    
    # Estadísticas
    print(f"📊 Estadísticas:")
    print(f"   - Score promedio: {np.mean(anomaly_scores):.3f}")
    print(f"   - Score mínimo (más anómalo): {np.min(anomaly_scores):.3f}")
    print(f"   - Anomalías detectadas: {np.sum(anomalies)} ({np.sum(anomalies)/len(data)*100:.1f}%)")
    
    # Visualización comparativa
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Distribución de scores
    axes[0, 0].hist(anomaly_scores, bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0, 0].set_xlabel('Anomaly Score')
    axes[0, 0].set_ylabel('Frecuencia')
    axes[0, 0].set_title('Distribución de Scores - Isolation Forest')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Comparación con otros métodos
    methods = ['Euclidiana', 'Mahalanobis', 'Isolation Forest']
    anomaly_counts = [
        np.sum(data['euclidean_anomaly']),
        np.sum(data['mahalanobis_anomaly']),
        np.sum(data['isolation_anomaly'])
    ]
    
    axes[0, 1].bar(methods, anomaly_counts, color=['red', 'yellow', 'green'], alpha=0.7)
    axes[0, 1].set_ylabel('Número de Anomalías')
    axes[0, 1].set_title('Comparación de Métodos')
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. Venn diagram conceptual - Overlap de métodos
    all_methods = data['euclidean_anomaly'] & data['mahalanobis_anomaly'] & data['isolation_anomaly']
    eucl_maha = data['euclidean_anomaly'] & data['mahalanobis_anomaly'] & ~data['isolation_anomaly']
    eucl_iso = data['euclidean_anomaly'] & ~data['mahalanobis_anomaly'] & data['isolation_anomaly']
    maha_iso = ~data['euclidean_anomaly'] & data['mahalanobis_anomaly'] & data['isolation_anomaly']
    
    overlap_data = {
        'Todos': np.sum(all_methods),
        'Eucl+Maha': np.sum(eucl_maha),
        'Eucl+Iso': np.sum(eucl_iso),
        'Maha+Iso': np.sum(maha_iso),
        'Solo Eucl': np.sum(data['euclidean_anomaly'] & ~data['mahalanobis_anomaly'] & ~data['isolation_anomaly']),
        'Solo Maha': np.sum(~data['euclidean_anomaly'] & data['mahalanobis_anomaly'] & ~data['isolation_anomaly']),
        'Solo Iso': np.sum(~data['euclidean_anomaly'] & ~data['mahalanobis_anomaly'] & data['isolation_anomaly'])
    }
    
    axes[1, 0].bar(overlap_data.keys(), overlap_data.values(), alpha=0.7)
    axes[1, 0].set_xlabel('Combinación de Métodos')
    axes[1, 0].set_ylabel('Número de Anomalías')
    axes[1, 0].set_title('Intersección de Detecciones')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # 4. Serie temporal con todos los métodos
    axes[1, 1].scatter(data['hour_rounded'], data['performance_ratio'],
                      c='blue', alpha=0.3, s=20, label='Normal')
    
    # Marcar anomalías de cada método
    for method, color, marker in [
        ('euclidean_anomaly', 'red', 'o'),
        ('mahalanobis_anomaly', 'yellow', 's'),
        ('isolation_anomaly', 'green', '^')
    ]:
        anomaly_data = data[data[method]]
        if len(anomaly_data) > 0:
            axes[1, 1].scatter(anomaly_data['hour_rounded'], 
                              anomaly_data['performance_ratio'],
                              c=color, s=50, marker=marker, 
                              label=method.replace('_anomaly', '').title(),
                              edgecolors='black', linewidths=0.5)
    
    axes[1, 1].set_xlabel('Fecha')
    axes[1, 1].set_ylabel('Performance Ratio')
    axes[1, 1].set_title('Anomalías Detectadas por Cada Método')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Fechas con anomalías
    anomaly_dates = data[data['isolation_anomaly']]['hour_rounded'].tolist()
    
    print(f"\n📅 Primeras 10 fechas con anomalías (Isolation Forest):")
    for i, date in enumerate(anomaly_dates[:10], 1):
        print(f"   {i}. {date}")
    
    # Análisis de sensibilidad a correlaciones
    only_mahalanobis = np.sum(~data['euclidean_anomaly'] & data['mahalanobis_anomaly'] & ~data['isolation_anomaly'])
    print(f"\n🔍 ANÁLISIS DE SENSIBILIDAD A CORRELACIONES:")
    print(f"\nMahalanobis detectó {only_mahalanobis} anomalías únicas")
    print("Esto sugiere que Mahalanobis es MÁS sensible a correlaciones porque:")
    print("- Considera la estructura de covarianza de los datos")
    print("- Detecta puntos que son anómalos en el contexto multivariado")
    print("- Identifica observaciones que rompen patrones de correlación normales")
    
    return data, anomaly_dates

# Aplicar Isolation Forest
hourly_data, isolation_anomalies = isolation_forest_detector(
    X_scaled, hourly_data.copy(), contamination=0.05
)


# ========================================
# CELDA 11: Análisis integrado y reporte final
# ========================================

In [ ]:
def generate_anomaly_report(data):
    """
    Genera reporte consolidado de anomalías detectadas
    """
    print("\n📊 REPORTE CONSOLIDADO DE ANOMALÍAS")
    print("="*70)
    
    # Crear score de severidad combinado
    data['anomaly_count'] = (
        data['euclidean_anomaly'].astype(int) +
        data['mahalanobis_anomaly'].astype(int) +
        data['isolation_anomaly'].astype(int)
    )
    
    # Normalizar scores para severidad (0-100)
    data['euclidean_severity'] = (data['euclidean_distance'] / 
                                  data['euclidean_distance'].max() * 100)
    data['mahalanobis_severity'] = (data['mahalanobis_distance'] / 
                                    data['mahalanobis_distance'].max() * 100)
    data['isolation_severity'] = ((1 - (data['isolation_score'] - 
                                       data['isolation_score'].min()) / 
                                  (data['isolation_score'].max() - 
                                   data['isolation_score'].min())) * 100)
    
    # Score de severidad promedio ponderado
    data['severity_score'] = (
        0.3 * data['euclidean_severity'] +
        0.4 * data['mahalanobis_severity'] +  # Mayor peso por sensibilidad a correlaciones
        0.3 * data['isolation_severity']
    )
    
    # Filtrar anomalías (al menos 1 método)
    anomalies = data[data['anomaly_count'] > 0].copy()
    anomalies = anomalies.sort_values('severity_score', ascending=False)
    
    # Top 20 anomalías más severas
    print(f"\n🚨 TOP 20 ANOMALÍAS MÁS SEVERAS:")
    print(f"{'Fecha':<20} {'Métodos':<10} {'Severidad':<10} {'PR':<8} {'Efic%':<8} {'Irrad':<8}")
    print("-"*70)
    
    for idx, row in anomalies.head(20).iterrows():
        methods = row['anomaly_count']
        date = row['hour_rounded'].strftime('%Y-%m-%d %H:%M')
        severity = row['severity_score']
        pr = row['performance_ratio']
        eff = row['system_efficiency']
        irr = row['poa_irradiance_o_149574']
        
        print(f"{date:<20} {methods:<10} {severity:>8.1f} {pr:>8.2f} {eff:>8.1f} {irr:>8.0f}")
    
    # Guardar lista completa
    anomaly_list = anomalies[['hour_rounded', 'anomaly_count', 'severity_score',
                              'euclidean_anomaly', 'mahalanobis_anomaly', 
                              'isolation_anomaly', 'performance_ratio',
                              'system_efficiency', 'total_ac_power']].copy()
    
    # Análisis por método
    print(f"\n📈 RESUMEN POR MÉTODO:")
    print(f"   - Euclidiana: {np.sum(data['euclidean_anomaly'])} anomalías")
    print(f"   - Mahalanobis: {np.sum(data['mahalanobis_anomaly'])} anomalías")
    print(f"   - Isolation Forest: {np.sum(data['isolation_anomaly'])} anomalías")
    print(f"   - Detectadas por 3 métodos: {np.sum(data['anomaly_count'] == 3)}")
    print(f"   - Detectadas por 2 métodos: {np.sum(data['anomaly_count'] == 2)}")
    print(f"   - Detectadas por 1 método: {np.sum(data['anomaly_count'] == 1)}")
    
    # Visualización final
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Timeline de severidad
    scatter = ax1.scatter(data['hour_rounded'], data['severity_score'],
                         c=data['anomaly_count'], cmap='YlOrRd', 
                         s=50, alpha=0.6)
    ax1.set_xlabel('Fecha')
    ax1.set_ylabel('Score de Severidad (0-100)')
    ax1.set_title('Timeline de Anomalías - Severidad y Consenso entre Métodos')
    ax1.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax1, label='Número de métodos')
    
    # Distribución de severidad por número de métodos
    for i in range(4):
        subset = data[data['anomaly_count'] == i]['severity_score']
        if len(subset) > 0:
            ax2.hist(subset, bins=30, alpha=0.6, 
                    label=f'{i} métodos', density=True)
    
    ax2.set_xlabel('Score de Severidad')
    ax2.set_ylabel('Densidad')
    ax2.set_title('Distribución de Severidad por Consenso de Métodos')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return anomaly_list


# ========================================
# CELDA 12: Respuestas finales y guardar resultados
# ========================================

In [ ]:
print("\n" + "="*70)
print("RESPUESTAS A LAS PREGUNTAS DEL ANÁLISIS")
print("="*70)

print("\n❓ ¿Qué método parece más sensible a las correlaciones?")
print("\n✅ RESPUESTA: La Distancia de Mahalanobis es el método MÁS SENSIBLE a las correlaciones")
print("\nJustificación:")
print("1. Mahalanobis considera la matriz de covarianza completa de los datos")
print("2. Detecta anomalías en el contexto multivariado (relaciones entre variables)")
print("3. Euclidiana trata todas las dimensiones independientemente")
print("4. Isolation Forest se enfoca en aislamiento estructural, no correlaciones directas")

only_mahalanobis = np.sum(~hourly_data['euclidean_anomaly'] & 
                          hourly_data['mahalanobis_anomaly'] & 
                          ~hourly_data['isolation_anomaly'])

print(f"\nEvidencia: Mahalanobis detectó {only_mahalanobis} anomalías únicas")
print("que los otros métodos no identificaron, sugiriendo sensibilidad a patrones")
print("de correlación anormales.")

# Guardar resultados
print("\n💾 Guardando resultados...")
hourly_data.to_csv('anomalias_detectadas_planta_solar.csv', index=False)
print("✅ Archivo guardado: anomalias_detectadas_planta_solar.csv")


# ========================================
# CELDA 13: Modelo ensamblado (diseño conceptual)
# ========================================

In [ ]:
print("\n" + "="*70)
print("DISEÑO DE MODELO ENSAMBLADO")
print("="*70)

print("""
📋 DIAGRAMA DE FLUJO - MODELO ENSAMBLADO DE DETECCIÓN DE ANOMALÍAS

┌─────────────────┐
│  Datos de       │
│  Entrada        │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ Preprocesamiento│
│ - Limpieza      │
│ - Normalización │
│ - Features      │
└────────┬────────┘
         │
    ┌────┴────┬─────────┬──────────┐
    ▼         ▼         ▼          │
┌────────┐┌────────┐┌──────────┐  │
│Euclidean││Mahalanobis││Isolation│  │
│Distance ││Distance   ││Forest   │  │
└────┬────┘└────┬────┘└────┬────┘  │
     │          │           │       │
     ▼          ▼           ▼       │
┌─────────────────────────────┐    │
│    Cálculo de Scores        │    │
│  - Score_E (0-1)           │    │
│  - Score_M (0-1)           │    │
│  - Score_I (0-1)           │    │
└──────────────┬──────────────┘    │
               │                    │
               ▼                    │
┌─────────────────────────────┐    │
│  ENSAMBLADO PONDERADO       │    │
│  Score_final = w1*Score_E + │    │
│                w2*Score_M + │    │
│                w3*Score_I   │    │
│  (w1=0.25, w2=0.45, w3=0.30)│    │
└──────────────┬──────────────┘    │
               │                    │
               ▼                    │
┌─────────────────────────────┐    │
│  MÉTRICA DE SEVERIDAD       │    │
│  Severity = Score_final × F │    │
│  F = factor contextual      │    │
└──────────────┬──────────────┘    │
               │                    │
               ▼                    │
┌─────────────────────────────┐    │
│  CLASIFICACIÓN FINAL        │    │
│  - Normal (S < 30)         │    │
│  - Leve (30 ≤ S < 50)     │    │
│  - Moderada (50 ≤ S < 70) │    │
│  - Severa (S ≥ 70)        │    │
└─────────────────────────────┘    │
                                   │
               ┌───────────────────┘
               ▼
┌─────────────────────────────┐
│  RETROALIMENTACIÓN          │
│  - Validación experta      │
│  - Ajuste de pesos         │
│  - Reentrenamiento         │
└─────────────────────────────┘
""")

# Ejemplo de implementación
def ensemble_anomaly_score(eucl_score, maha_score, iso_score, 
                          hour, irradiance, affected_inverters):
    """
    Calcula score de anomalía ensamblado con factor contextual
    """
    # Pesos base
    w1, w2, w3 = 0.25, 0.45, 0.30
    
    # Score ponderado base
    base_score = w1 * eucl_score + w2 * maha_score + w3 * iso_score
    
    # Factor contextual
    night_factor = 2.0 if hour < 6 or hour > 20 else 1.0
    low_irr_factor = 1.5 if irradiance < 100 else 1.0
    multi_inv_factor = 1 + (affected_inverters / 24)
    
    # Score final
    contextual_factor = night_factor * low_irr_factor * multi_inv_factor
    final_score = min(100, base_score * contextual_factor)
    
    return final_score

print("\n✅ Análisis completado exitosamente!")
print("="*70)